In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
from statsmodels.stats.multicomp import pairwise_tukeyhsd


Провести дисперсионный анализ для определения того, есть ли различия среднего роста среди взрослых футболистов, хоккеистов и штангистов. Даны значения роста в трех группах случайно выбранных спортсменов: Футболисты: 173, 175, 180, 178, 177, 185, 183, 182. Хоккеисты: 177, 179, 180, 188, 177, 172, 171, 184, 180. Штангисты: 172, 173, 169, 177, 166, 180, 178, 177, 172, 166, 170

In [ ]:
f = np.array([173, 175, 180, 178, 177, 185, 183, 182])
h = np.array([177, 179, 180, 188, 177, 172, 171, 184, 180])
w = np.array([172, 173, 169, 177, 166, 180, 178, 177, 172, 166, 170])

Прежде всего проведем проверки на нормальность и равенство дисперсий.


In [ ]:
stats.shapiro(f)

ShapiroResult(statistic=0.9775082468986511, pvalue=0.9495404362678528)

In [ ]:
stats.shapiro(h)

ShapiroResult(statistic=0.9579196572303772, pvalue=0.7763139009475708)

In [ ]:
stats.shapiro(w)

ShapiroResult(statistic=0.9386808276176453, pvalue=0.5051165223121643)

Все значения в выборках распределены нормально.

In [ ]:
stats.bartlett(f, h, w)

BartlettResult(statistic=0.4640521043406442, pvalue=0.7929254656083131)

Равенство дисперсий тоже соблюдено. Проведем анализ.

In [ ]:
k = 3
f_mean = np.mean(f)
f_mean

179.125

In [ ]:
h_mean = np.mean(h)
h_mean

178.66666666666666

In [ ]:
w_mean = np.mean(w)
w_mean

172.72727272727272

In [ ]:
total = np.array([*f, *h, *w])
total

array([173, 175, 180, 178, 177, 185, 183, 182, 177, 179, 180, 188, 177,
       172, 171, 184, 180, 172, 173, 169, 177, 166, 180, 178, 177, 172,
       166, 170])

In [ ]:
n = len(total)
n

28

In [ ]:
total_mean = np.mean(total)
total_mean

176.46428571428572

In [ ]:
S = np.sum((total - total_mean)**2)
S

830.9642857142854

In [ ]:
S_f = np.sum((f_mean - total_mean)**2)* len(f) + np.sum((h_mean - total_mean)**2)* len(h) + np.sum((w_mean - total_mean)**2)* len(w)
S_f

253.9074675324678

In [ ]:
S_ost = np.sum((f - f_mean)**2) + np.sum((h - h_mean)**2) + np.sum((w - w_mean)**2)
S_ost

577.0568181818182

In [ ]:
S_ost + S_f

830.964285714286

Сумма равна S, Значит расчеты произведены верно.

In [ ]:
D_f = S_f / (k-1)
D_f

126.9537337662339

In [ ]:
D_ost = S_ost / (n-k)
D_ost

23.08227272727273

In [ ]:
F_n = D_f / D_ost
F_n

5.500053450812598

Зафиксируем уровень значимости alpha = 0,05, для него найдем Fcrit.

In [ ]:
alpha = 0.05
F_crit = stats.f.ppf(1-alpha, k-1, n-k)
F_crit

3.3851899614491687

Как видим, F_n > F_crit, а значит статистически значимые различия в росте выборок спортсменов имеются.

In [ ]:
f = stats.f_oneway(f, h, w)
f

F_onewayResult(statistic=5.500053450812596, pvalue=0.010482206918698694)

p-value < alpha, что подтверждает различия в среднем росте спортсменов по выборкам.

Данную задачу можно было решеить и другим способом.

In [ ]:
stats.kruskal(f, h, w)

KruskalResult(statistic=9.207658779576581, pvalue=0.010013416954688928)

p-value < alpha, значит стат.значимые различия в росте имеются.

In [ ]:
df = pd.DataFrame({"score": [173, 175, 180, 178, 177, 185, 183, 182,
                             177, 179, 180, 188, 177, 172, 171, 184, 180,
                             172, 173, 169, 177, 166, 180, 178, 177, 172, 166, 170],
                   "group": np.repeat(["f", "h", "w"], repeats = [8, 9, 11])})

In [ ]:
tukey = pairwise_tukeyhsd(df["score"], df["group"], alpha = 0.05)
print(tukey)

 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower    upper  reject
-----------------------------------------------------
     f      h  -0.4583  0.979  -6.2732  5.3566  False
     f      w  -6.3977 0.0219 -11.9583 -0.8372   True
     h      w  -5.9394 0.0284 -11.3181 -0.5607   True
-----------------------------------------------------


Из таблицы видно, что стат.значимых различий в росте нет только между футболистами и хоккеистами. Между штангистами и остальными выборками стат.значимые различия в росте спортсменов имеются.